In [1]:
import warnings
warnings.simplefilter('ignore')

import csv
import string
import json
import itertools
import random
import collections
import YelpCleanData as cd

import pandas as pd
import numpy as np

from wordcloud import WordCloud
from wordcloud import STOPWORDS
from nltk.util import ngrams

In [2]:
random.seed(9)

In [3]:
user_business_df = cd.load_data('data/rating_history.csv')
cat_split_city = user_business_df['categories'].apply(lambda x: list(set(list(map(str.strip, x.split(','))))) if x is not None else 'None')
user_business_df['categories'] = cat_split_city
cat_merged = set(itertools.chain(*list(cat_split_city.values)))

In [5]:
business = cd.open_json('data/business.json')
df_business = pd.json_normalize(business)

In [6]:
cat_split_entire = df_business['categories'].apply(lambda x: list(set(list(map(str.strip, x.split(','))))) if x is not None else 'None')
df_business['categories'] = cat_split_entire

In [7]:
model = cd.build_word2vec(cat_split_entire, window=20, sg=0, negative=20, alpha=0.03, min_alpha=0.001, epochs=20)

In [8]:
# Mapping StockCode with similarity scores with respect to categories existing in the prediction set
category = 'Brazilian'
similar_df = cd.get_similar_cat_df(model, category)
similar_df = similar_df[similar_df['Category'].isin(cat_merged)].reset_index(drop=True)
similar_df.head()

,Category,Similarity
0,Brazilian,1.000000
1,Argentine,0.671038
2,Peruvian,0.586984
3,Basque,0.512819
4,Seafood,0.491764


In [9]:
def unique(list1):
    # insert the list to the set
    list_set = set(list1)
    # convert the set to the list
    unique_list = (list(list_set))
    return unique_list

high_ratings = cd.load_data('data/high_ratings.csv')
high_ratings_busi = unique(high_ratings['business_id'])
len(high_ratings_busi)

3330

In [53]:
#Index 541, 3014 has error.
remove_val = high_ratings_busi[541], high_ratings_busi[3014]
high_ratings_busi.remove(remove_val[0])
high_ratings_busi.remove(remove_val[1])

In [56]:
# Find stop words of potential positive reviews (high_rating.csv) 
high_ratings_word_list = []
high_ratings_2_word_list = []
for b_id in high_ratings_busi:
    if __name__ == "__main__":
        refer_reviews_df = cd.get_refer_table(b_id, isHighRatings=True).reset_index(drop=True)
    
        b_cat = user_business_df[user_business_df['business_id'] == b_id]['categories'].iloc[0][0]
        similar_cat = cd.get_n_similar_cat(model, b_cat, cat_merged, n=5)
    
        ref_df = cd.get_potential_reviewers_history_from_today(user_business_df, refer_reviews_df, similar_cat, year_diff=5, isHigh=True)
        nlp_df = cd.get_most_recent_reviews(ref_df)
        text_series = cd.text_cleaning(nlp_df['text'], n_gram=1)
        text_series_2_word = cd.text_cleaning(nlp_df['text'], n_gram=2)
        for item in text_series:
            high_ratings_word_list += item.split()
        for item in text_series_2_word:
            high_ratings_2_word_list += item.split()

In [105]:
import collections
high_ratings_word_counts = collections.Counter(high_ratings_word_list)
high_ratings_word_counts_top_200 = high_ratings_word_counts.most_common(200)
print(high_ratings_word_counts_top_200)

[('food', 77905), ('great', 76236), ('place', 64643), ('good', 64161), ('delicious', 46521), ('like', 44903), ('time', 42381), ('really', 41930), ('just', 38563), ('definitely', 34978), ('service', 34747), ('amazing', 34417), ('menu', 33810), ('best', 32216), ('love', 32027), ('try', 31401), ('got', 28712), ('ive', 28323), ('chicken', 27004), ('im', 26627), ('spot', 26500), ('ordered', 25517), ('nice', 24976), ('restaurant', 24871), ('bar', 24710), ('order', 23968), ('friendly', 23482), ('cheese', 23102), ('staff', 21958), ('recommend', 21306), ('city', 20946), ('pizza', 19781), ('perfect', 19715), ('favorite', 19711), ('fresh', 19412), ('come', 19164), ('super', 18996), ('came', 18625), ('philly', 18190), ('wait', 17540), ('sauce', 17355), ('dinner', 17300), ('happy', 16607), ('youre', 16520), ('drinks', 16181), ('experience', 16070), ('awesome', 15633), ('salad', 15232), ('little', 15116), ('lunch', 14403), ('night', 14335), ('new', 14255), ('make', 13817), ('dont', 13808), ('meal', 

In [111]:
import collections
high_ratings_2_word_counts = collections.Counter(high_ratings_2_word_list)
high_ratings_2_word_counts_top_100 = high_ratings_2_word_counts.most_common(100)
print(high_ratings_2_word_counts_top_100)

[('highly_recommend', 10265), ('happy_hour', 9422), ('ice_cream', 6805), ('really_good', 6614), ('staff_friendly', 5167), ('great_food', 4311), ('looking_forward', 4280), ('great_service', 4227), ('love_place', 4202), ('center_city', 3984), ('service_great', 3801), ('great_place', 3793), ('super_friendly', 3665), ('5_stars', 3545), ('fried_chicken', 3280), ('food_good', 3150), ('make_sure', 2884), ('outdoor_seating', 2814), ('food_great', 2774), ('looks_like', 2738), ('friendly_staff', 2735), ('customer_service', 2694), ('im_glad', 2617), ('breakfast_lunch', 2597), ('mac_cheese', 2577), ('great_spot', 2550), ('super_nice', 2538), ('great_experience', 2532), ('wait_come', 2530), ('absolutely_amazing', 2526), ('double_knot', 2497), ('really_cool', 2438), ('south_philly', 2351), ('definitely_recommend', 2339), ('really_nice', 2298), ('good_food', 2274), ('art_deco', 2214), ('ive_tried', 2197), ('forward_coming', 2194), ('feels_like', 2171), ('university_city', 2115), ('definitely_worth', 

In [60]:
low_ratings = cd.load_data('data/low_ratings.csv')
low_ratings_busi = unique(low_ratings['business_id'])
len(low_ratings_busi)

1254

In [61]:
# Find stop words of potential negative reviews (low_rating.csv) #index  has error
low_ratings_word_list = []
low_ratings_2_word_list = []
for b_id in low_ratings_busi:
    if __name__ == "__main__":
        refer_reviews_df = cd.get_refer_table(b_id, isHighRatings=False).reset_index(drop=True)
    
        b_cat = user_business_df[user_business_df['business_id'] == b_id]['categories'].iloc[0][0]
        similar_cat = cd.get_n_similar_cat(model, b_cat, cat_merged, n=5)
    
        ref_df = cd.get_potential_reviewers_history_from_today(user_business_df, refer_reviews_df, similar_cat, year_diff=5, isHigh=False)
        nlp_df = cd.get_most_recent_reviews(ref_df)
        text_series = cd.text_cleaning(nlp_df['text'], n_gram=1)
        text_series_2_word = cd.text_cleaning(nlp_df['text'], n_gram=2)
        for item in text_series:
            low_ratings_word_list += item.split()
        for item in text_series_2_word:
            low_ratings_2_word_list += item.split()

In [114]:
import collections
low_ratings_word_counts = collections.Counter(low_ratings_word_list)
low_ratings_word_counts_top_100 = low_ratings_word_counts.most_common(100)
print(low_ratings_word_counts_top_100)

[('food', 32069), ('order', 24604), ('just', 22039), ('place', 21312), ('like', 18191), ('ordered', 17124), ('time', 16504), ('dont', 15660), ('good', 13493), ('chicken', 13435), ('restaurant', 13321), ('came', 13264), ('got', 12501), ('service', 10607), ('did', 10526), ('really', 10473), ('didnt', 9967), ('people', 9937), ('said', 9674), ('im', 9198), ('know', 9001), ('minutes', 8937), ('table', 8774), ('cheese', 8417), ('told', 8232), ('went', 8179), ('eat', 7673), ('pizza', 7541), ('going', 7492), ('took', 7487), ('come', 6574), ('2', 6545), ('better', 6309), ('asked', 6287), ('ive', 6150), ('bad', 6000), ('money', 5942), ('menu', 5929), ('say', 5857), ('flavor', 5821), ('try', 5667), ('make', 5595), ('wasnt', 5557), ('want', 5398), ('way', 5338), ('wait', 5231), ('cold', 5167), ('server', 4981), ('wanted', 4956), ('experience', 4906), ('night', 4850), ('sauce', 4686), ('dry', 4534), ('salad', 4534), ('fries', 4480), ('times', 4461), ('left', 4385), ('sandwich', 4380), ('day', 4369)

In [127]:
import collections
low_ratings_2_word_counts = collections.Counter(low_ratings_2_word_list)
low_ratings_2_word_counts_top_100 = low_ratings_2_word_counts.most_common(100)
print(low_ratings_2_word_counts_top_100)

[('dont_know', 2453), ('grub_hub', 2444), ('customer_service', 1796), ('n_cheese', 1785), ('mac_n', 1766), ('crab_legs', 1493), ('ice_cream', 1490), ('tip_cash', 1416), ('chicken_cheesesteak', 1381), ('seafood_salad', 1352), ('10_minutes', 1343), ('20_minutes', 1290), ('tasted_like', 1226), ('fried_chicken', 1220), ('waste_money', 1142), ('french_75', 1074), ('got_food', 1063), ('intended_tip', 1035), ('30_minutes', 989), ('mac_cheese', 950), ('late_night', 934), ('butter_cake', 915), ('osso_bucco', 888), ('placed_order', 882), ('time_ordering', 863), ('good_pizza', 832), ('people_waiting', 825), ('did_want', 817), ('really_dont', 815), ('dont_care', 796), ('didnt_want', 777), ('1_star', 774), ('15_minutes', 760), ('taste_like', 753), ('center_city', 752), ('dont_think', 734), ('red_velvet', 732), ('stopped_going', 725), ('velvet_cheesecake', 717), ('grand_marnier', 716), ('minutes_later', 711), ('time_order', 711), ('3_times', 699), ('ive_seen', 676), ('long_time', 675), ('little_bit'

In [ ]:
# We find that the quality of food and customer service appear frequently in both low-rating and high-rating reviews, thus we 
# remove related words from wordcloud.

In [138]:
high_rating_stopwords = ['great', 'place', 'good', 'delicious', 'like', 'time', 'really', 'just', 'definitely', 
                         'amazing', 'best', 'love', 'try', 'got', 'ive', 'im', 'spot', 'ordered', 'nice', 
                         'restaurant', 'order', 'recommend', 'city', 'perfect', 'favorite', 'come', 'super', 'came',
                         'philly', 'happy', 'youre', 'experience', 'awesome', 'little', 'make', 'dont', 'meal', 
                         'loved', 'right', 'highly', 'say', 'way', 'area', 'tried', 'absolutely', 'table', 'eat', 
                         'went', 'excellent', 'day', 'hour', 'fantastic', 'different', 'want', 'sure', 
                         'philadelphia', 'looking', 'better', 'worth', 'bit', 'lot', 'date', 'coming', 'didnt', 
                         'things', 'friend', 'times', 'going', 'know', 'people', 'did', 'perfectly', 'pretty', 
                         'feel', '5', 'check', 'extremely', 'dishes', 'dining', 'think', 'items', 'served', 'away',
                         'stars', 'especially', 'enjoy', 'soon', 'week', 'wonderful', 'eating', 'point', '2', 
                         'years','stop', 'enjoyed', 'forward', 'need', 'ordering', 'kind', 'wanted', 'thats',
                         'highly_recommend', 'really_good', 'looking_forward', 'love_place', 'great_place',
                         '5_stars', 'make_sure', 'looks_like', 'im_glad', 'great_spot', 'super_nice', 
                         'absolutely_amazing', 'really_cool', 'definitely_recommend', 'really_nice', 'ive_tried',
                         'forward_coming', 'feels_like', 'definitely_try', 'time_im', 'really_enjoyed', 'perfect_spot',
                         'definitely_come', 'highly_recommended', 'just_right', 'look_forward', 'place_really', 
                         'im_sure', 'youre_looking', 'come_try', 'really_great', 'dining_experience', 
                         'amazing_experience', 'spot_ive']
high_series = pd.Series(high_rating_stopwords, name = 'Word')
high_series.to_csv('high_rating_stopwords.csv',index=False)

In [139]:
low_rating_stopwords = ['just', 'place', 'like', 'ordered', 'time', 'dont', 'good', 'restaurant', 'came', 'got', 
                        'did', 'really', 'didnt', 'people', 'said', 'im', 'know', 'minutes', 'table', 'told', 'went',
                        'eat', 'going', 'took', 'say', 'try', 'make', 'wasnt', 'want', 'way', 'wanted', 'times', 
                        'day', 'gave', 'great', 'ask', '10', 'called', 'decided', 'ok', '3', 'right', 'tried', 
                        'thats', 'coming', 'person', 'away', 'hour', 'tasted_like', 'got_food', 'placed_order', 
                        '1_star', 'taste_like', 'stopped_going', 'ive_seen', 'little_bit', 'im_really', 'food_came', 
                        'im_sure', 'flavor_just', 'looked_like', 'look_like', 'decided_try', 'order_ordered', 
                        'order_didnt', 'people_actually', 'said_yes', 'came_tried', 'understand_got']
low_series = pd.Series(low_rating_stopwords, name = 'Word')
low_series.to_csv('low_rating_stopwords.csv',index=False)